# Propose centroid replacing the masters with BERT siamese

In [1]:
# import tensorflow as tf
import keras
# from tensorflow.python import keras
import os

Using TensorFlow backend.


In [2]:
# Optimize the use of GPUs
# https://datascience.stackexchange.com/questions/23895/multi-gpu-in-keras
# https://keras.io/getting-started/faq/#how-can-i-run-a-keras-model-on-multiple-gpus
# https://stackoverflow.com/questions/56316451/how-to-use-specific-gpus-in-keras-for-multi-gpu-training

In [3]:
from __future__ import print_function, division

In [4]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [5]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras import optimizers

In [6]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [7]:
MAX_SEQUENCE_LENGTH_T = 20 # 20
MAX_SEQUENCE_LENGTH_D = 20 # 80
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

'''
    Configuration
'''
epochs = 1000
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

### Parse bugs preproprecessed

In [8]:
# Domain to use
DOMAIN = 'eclipse'
METHOD = 'deepQL_weights_{}'.format(epochs)
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = '{}_feature@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [9]:
# !wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
# !unzip -o uncased_L-12_H-768_A-12.zip

In [10]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [11]:
from keras_bert import load_vocabulary

token_dict = load_vocabulary(vocab_path)

In [12]:
"Total vocabulary: {}".format(len(token_dict))

'Total vocabulary: 30522'

In [13]:
baseline = Baseline(DOMAIN, DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 
                    token_dict['[CLS]'], token_dict['[SEP]'])
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [14]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [15]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


361006

#### Dicionário de títulos e descrições

In [16]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 30.3 s, sys: 3.2 s, total: 33.5 s
Wall time: 33.3 s


#### Hashing bugs by buckets

In [17]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [18]:
%%time

experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

Reading train data
Reading bug ids
CPU times: user 2min 36s, sys: 23 ms, total: 2min 36s
Wall time: 2min 36s


#### Recovery bug ids from train

In [19]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

#### Display a random bug

In [20]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '2\n',
 'bug_status': '0\n',
 'component': '461\n',
 'creation_ts': '2012-12-06 10:56:00 -0500',
 'delta_ts': '2012-12-07 11:30:06 -0500',
 'description': "[CLS] created attachment 224 ##38 ##1 screens ##hot of empty theme pop ##up i tried the new editor co ##g but there are no themes offered . i just get some gray pixels ( border of an empty pop ##up ) . note i haven ' t chosen a theme before , so maybe that ' s the problem . we need to ensure that a theme ##less user gets the right behavior since that is the default state . [SEP]",
 'description_bert': "[CLS] created attachment 224 ##38 ##1 screens ##hot of empty theme pop ##up i tried the new editor co ##g but there are no themes offered . i just get some gray pixels ( border of an empty pop ##up ) . note i haven ' t chosen a theme before , so maybe that ' s the problem . we need to ensure that a theme ##less user gets the right behavior since that is the default state . [SEP]",
 'description_word': array([  101,  2

### Generating the batch test

In [21]:
"Train ", len(baseline.dup_sets_train)

('Train ', 34882)

In [22]:
bug_idx = bug_train_ids[0]
vector = baseline.bug_set[bug_idx]['textual_word']
annoy_train = AnnoyIndex(vector.shape[0])
for bug_id in bug_train_ids:
    annoy_train.add_item(bug_id, baseline.bug_set[bug_id]['textual_word'])
annoy_train.build(10) # 10 trees
"Indexed all train"

'Indexed all train'

### Train ids

In [23]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

In [24]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, \
                            valid_master_sample, valid_master_neg, valid_sim = experiment.batch_iterator_bert(None, baseline.train_data, 
                                                                                          baseline.dup_sets_train,
                                                                                          bug_train_ids,
                                                                                          batch_size_test, 1, 
                                                                                              issues_by_buckets, INCLUDE_MASTER=True)

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title']['token'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description']['token'].shape[1]

CPU times: user 65.4 ms, sys: 29 µs, total: 65.4 ms
Wall time: 65 ms


In [25]:
valid_input_sample['title']['token'].shape, valid_input_sample['description']['token'].shape, \
    valid_input_sample['info'].shape, valid_sim.shape

((128, 20), (128, 20), (128, 1682), (128,))

### Validar entrada

In [26]:
# %%time 

# baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5, batch_iterator, issues_by_buckets)

## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [27]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### BERT

https://github.com/CyberZHG/keras-bert

In [28]:
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import compile_model, get_model
from keras.layers import GlobalAveragePooling1D

def bert_model(MAX_SEQUENCE_LENGTH, name):
    layer_num = 8
#     model = load_trained_model_from_checkpoint(
#             config_path,
#             model_path,
#             training=True,
#             trainable=True,
#             seq_len=MAX_SEQUENCE_LENGTH,
#     )
    model = load_trained_model_from_checkpoint(
        config_path,
        model_path,
        training=True,
        use_adapter=True,
        seq_len=MAX_SEQUENCE_LENGTH,
        trainable=['Encoder-{}-MultiHeadSelfAttention-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-MultiHeadSelfAttention-Norm'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Norm'.format(i + 1) for i in range(layer_num)],
    )
#     model = get_model(
#         token_num=len(token_dict),
#         head_num=10,
#         transformer_num=layer_num,
#         embed_dim=100,
#         feed_forward_dim=100,
#         seq_len=MAX_SEQUENCE_LENGTH,
#         pos_num=MAX_SEQUENCE_LENGTH,
#         dropout_rate=0.05,
#     )
    compile_model(model)
    inputs = model.inputs[:2]
    outputs = model.get_layer('Encoder-{}-FeedForward-Norm'.format(layer_num)).output
    #outputs = model.get_layer('Extract').output
    outputs = GlobalAveragePooling1D()(outputs)
#     outputs = Dense(300, activation='tanh')(outputs)
    
    model = Model(inputs, outputs, name='FeatureBERTGenerationModel{}'.format(name))
    
    return model

### MLP

In [29]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    #layer = GRU(100, activation='tanh')(layer)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [30]:
from keras import backend as K
from keras.layers import Layer

'''
    Some loss ideas
    hinge loss Kullback-Leibler
    https://stackoverflow.com/questions/53581298/custom-combined-hinge-kb-divergence-loss-function-in-siamese-net-fails-to-genera
'''

def normalize(x, axis):
    norm = K.sqrt(K.sum(K.square(x), axis=axis, keepdims=False))
    return x, K.maximum(norm, K.epsilon())
    
# https://github.com/keras-team/keras/issues/3031
# https://github.com/keras-team/keras/issues/8335
def cosine_distance(inputs):
    x, y = inputs
    x, x_norm = normalize(x, axis=-1)
    y, y_norm = normalize(y, axis=-1)
    distance = K.sum( x * y, axis=-1) / (x_norm * y_norm)
    distance = (distance + K.constant(1)) / K.constant(2)
    # Distance goes from 0 to 2 in theory, but from 0 to 1 if x and y are both
    # positive (which is the case after ReLU activation).
    return K.mean(distance, axis=-1, keepdims=True)
    #return K.mean(distance, axis=-1, keepdims=False)

def _triplet_loss(y_true, y_pred):
    margin = K.constant(1.0)
    pos = y_pred[0]
    neg = y_pred[1]
    return K.mean(K.maximum(0.0, pos - neg + margin))

def triplet_loss(vects):
    pos = vects[0]
    neg = vects[1]
    margin = K.constant(1.0)
    return K.maximum(0.0, margin - pos + neg)
    #return K.mean(K.maximum(0.0, margin - pos + neg), keepdims=False)

class QuintetWeights(Layer):
 
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(QuintetWeights, self).__init__(**kwargs)
 
    def build(self, input_shape):
        self.W= self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        self.built = False 
 
    def call(self, x):
        return K.dot(x, self.W)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

class Triplet(Layer):
 
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Triplet, self).__init__(**kwargs)
 
    def build(self, input_shape):
        pass
 
    def call(self, x):
        pos, neg  = x
        margin = K.constant(1.0)
        return K.maximum(0.0, margin - pos + neg)
    
def custom_loss(y_true, y_pred):
    return y_pred[0][0]

def triplet_bug(y_true, y_pred):
    return y_pred[0][1]
def triplet_anchor(y_true, y_pred):
    return y_pred[0][2]
def triplet_pos(y_true, y_pred):
    return y_pred[0][3]
def triplet_neg(y_true, y_pred):
    return y_pred[0][4]

def pos_distance(y_true, y_pred):
    return y_pred[0]

def neg_distance(y_true, y_pred):
    return y_pred[1]

def stack_tensors(vects):
    return K.stack(vects, axis=-1)

# https://www.kaggle.com/c/quora-question-pairs/discussion/33631
# https://www.researchgate.net/figure/Illustration-of-triplet-loss-contrastive-loss-for-negative-samples-and-binomial_fig2_322060548
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    pos = y_pred[0]
    neg = y_pred[1]
    margin = 1
    return K.mean(pos * K.square(neg) +
                  (1 - pos) * K.square(K.maximum(margin - neg, 0)))

### Propose

In [31]:
from keras.layers import concatenate, Add, Lambda, Average, Maximum, Subtract, Average, AveragePooling1D, GlobalAveragePooling1D
from keras.optimizers import Adam, Nadam

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
    
    # Title
    bug_t_token = Input(shape = (sequence_length_t, ), name = 'title_token_{}'.format(name))
    bug_t_segment = Input(shape = (sequence_length_t, ), name = 'title_segment_{}'.format(name))
    # Description
    bug_d_token = Input(shape = (sequence_length_d, ), name = 'desc_token_{}'.format(name))
    bug_d_segment = Input(shape = (sequence_length_d, ), name = 'desc_segment_{}'.format(name))
    # Categorical
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model([bug_t_token, bug_t_segment])
    bug_d_feat = desc_feature_model([bug_d_token, bug_d_segment])
    bug_i_feat = categorical_feature_model(bug_i)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_t_token, bug_t_segment, bug_d_token, bug_d_segment, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [32]:
def siamese_model_centroid(sequence_length, name):
    
    # Bug Centroid Feature
    bug_centroid = Input(shape = (sequence_length, ), name = 'bug_centroid_feature_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_centroid], outputs=[bug_centroid], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [46]:
from keras.layers import Average, Reshape, Add
from keras_radam import RAdam
from keras_bert import AdamWarmup, calc_train_steps

def max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, 
                             master_anchor, master_negative, master_positive, 
                         NUMBER_OF_INSTANCES, BATCH_SIZE, EPOCHS, decay_lr=1):
    
    inputs = np.concatenate([encoded_anchor.input, encoded_positive.input, encoded_negative.input], -1).tolist()
    
    inputs.append(master_anchor.input)
    inputs.append(master_positive.input)
    inputs.append(master_negative.input)
    
    encoded_anchor = encoded_anchor.output
    encoded_positive = encoded_positive.output
    encoded_negative = encoded_negative.output
    master_anchor = master_anchor.output
    master_negative = master_negative.output
    master_positive = master_positive.output
    
    # Distance bugs
    positive_d = Lambda(cosine_distance, name='pos_cosine_distance')([encoded_anchor, encoded_positive])
    negative_d = Lambda(cosine_distance, name='neg_cosine_distance')([encoded_anchor, encoded_negative])
    
    # Distance masters anchor
    master_anchor_positive_d = Lambda(cosine_distance, name='pos_master_cosine_distance')([encoded_anchor, master_positive])
    master_anchor_negative_d = Lambda(cosine_distance, name='neg_master_cosine_distance')([encoded_anchor, master_negative])
    
    # Distance master positive
    master_pos_positive_d = Lambda(cosine_distance, name='pos_master_pos_cosine_distance')([encoded_positive, master_positive])
    master_pos_negative_d = Lambda(cosine_distance, name='neg_master_pos_cosine_distance')([encoded_positive, master_negative])
    
    # Distance master negative
    master_neg_positive_d = Lambda(cosine_distance, name='pos_master_neg_cosine_distance')([encoded_negative, master_negative])
    master_neg_negative_d = Lambda(cosine_distance, name='neg_master_neg_cosine_distance')([encoded_negative, master_positive])
     
#     output_bug = Triplet(1)([positive_d, negative_d])
#     output_bug = QuintetWeights((1,2))(output_bug)
    output_TL = Lambda(triplet_loss, name='triplet_pos_neg')([positive_d, negative_d])
    output_TL_a = Lambda(triplet_loss, name='triplet_anchor_centroid')([master_anchor_positive_d, master_anchor_negative_d])
    output_TL_p = Lambda(triplet_loss, name='triplet_pos_centroid')([master_pos_positive_d, master_pos_negative_d])
    output_TL_n = Lambda(triplet_loss, name='triplet_neg_centroid')([master_neg_positive_d, master_neg_negative_d])
    
    # Weights for each loss
    loss_TL = Reshape((1, ))(output_TL)
    loss_TL_a = Reshape((1, ))(output_TL_a)
    loss_TL_p = Reshape((1, ))(output_TL_p)
    loss_TL_n = Reshape((1, ))(output_TL_n)
    
    sum_all_losses = Add()([loss_TL, loss_TL_a, loss_TL_p, loss_TL_n])
    output_TL_w = Dense(1, input_shape=(1,), use_bias=False)(loss_TL)
    output_TL_a_w = Dense(1, input_shape=(1,), use_bias=False)(loss_TL_a)
    output_TL_p_w = Dense(1, input_shape=(1,), use_bias=False)(loss_TL_p)
    output_TL_n_w = Dense(1, input_shape=(1,), use_bias=False)(loss_TL_n)
    sum_all_loss_w = Add()([output_TL_w, output_TL_a_w, output_TL_p_w, output_TL_n_w])
    # Weighted Median Loss
    output_median = Lambda(
            lambda inputs: inputs[0] / inputs[1], 
            name = 'Weighted_Median_Loss')([sum_all_loss_w, sum_all_losses])
    
    output = concatenate([output_median, loss_TL, loss_TL_a, loss_TL_p, loss_TL_n])

    similarity_model = Model(inputs = inputs, outputs = [output], name = 'Similarity_Model')

    # setup the optimization process 
    similarity_model.compile(optimizer='adam',
                             metrics=[triplet_bug, triplet_anchor, triplet_pos, triplet_neg],
                             loss=custom_loss)

    # metrics=[triplet_bug, triplet_anchor, triplet_pos, triplet_neg],
    return similarity_model

In [47]:
%%time
import os

print("Batch size ", batch_size)

# Inspired on https://'pastebin.com/TaGFdcBA
keras.backend.clear_session()

# Feature models
'''
    cnn_dilated_model
    arcii_model
    cnn_model
    lstm_model
    bilstm_model
'''
# title_feature_model = bilstm_model(title_embedding_layer, MAX_SEQUENCE_LENGTH_T)
title_feature_model = bert_model(MAX_SEQUENCE_LENGTH_T, 'Title')
desc_feature_model = bert_model(MAX_SEQUENCE_LENGTH_D, 'Description')
#desc_feature_model = cnn_model(desc_embedding_layer, MAX_SEQUENCE_LENGTH_D)
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')
encoded_positive = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'pos')
encoded_negative = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'neg')
# Master model
embed_size = K.int_shape(title_feature_model.get_output_at(0))[1] + K.int_shape(desc_feature_model.get_output_at(0))[1] + K.int_shape(categorical_feature_model.get_output_at(0))[1] 

master_anchor = siamese_model_centroid(embed_size, 'master_anchor')
master_pos = siamese_model_centroid(embed_size, 'master_pos')
master_negative = siamese_model_centroid(embed_size, 'master_neg')

NUMBER_OF_INSTANCES = len(baseline.dup_sets_train)
BATCH_SIZE = batch_size
EPOCHS = epochs

similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, 
                                            master_anchor, master_negative, master_pos,
                                            NUMBER_OF_INSTANCES, BATCH_SIZE, EPOCHS, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()

'''
    Experiment
'''
for epoch in range(epochs):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, train_master_input, train_master_neg, \
            train_sim = experiment.batch_iterator_bert(encoded_anchor, baseline.train_data, baseline.dup_sets_train, \
                                                       bug_train_ids, 
                                                       batch_size, 1, 
                                                       issues_by_buckets, 
                                                       TRIPLET_HARD=True, USE_CENTROID=True)
    
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], train_input_sample['description']['token'], train_input_sample['description']['segment'], train_input_sample['info'],
                   train_input_pos['title']['token'], train_input_pos['title']['segment'], train_input_pos['description']['token'], train_input_pos['description']['segment'], train_input_pos['info'], 
                   train_input_neg['title']['token'], train_input_neg['title']['segment'], train_input_neg['description']['token'], train_input_neg['description']['segment'], train_input_neg['info'],
                   train_master_input['centroid_embed'],
                   train_master_input['centroid_embed'],
                   train_master_neg['centroid_embed']]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, _, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert')
        print("Epoch: {} Loss: {:.2f}, TL: {:.2f}, TL_A: {:.2f}, TL_P: {:.2f}, TL_N: {:.2f}, recall@25: {:.2f}".format(epoch+1, h[0], h[1], h[2], h[3], h[4], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, TL: {:.2f}, TL_A: {:.2f}, TL_P: {:.2f}, TL_N: {:.2f}".format(epoch+1,h[0], h[1], h[2], h[3], h[4]))
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}s, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

Batch size  64
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 1682)         0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 20)           0                                            
______________________________________________________________________________________________

Epoch: 1 Loss: 0.38, TL: 1.02, TL_A: 0.97, TL_P: 0.96, TL_N: 0.97
Epoch: 2 Loss: 0.38, TL: 1.02, TL_A: 0.97, TL_P: 0.96, TL_N: 0.97
Epoch: 3 Loss: 0.38, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.97
Epoch: 4 Loss: 0.38, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.97
Epoch: 5 Loss: 0.38, TL: 1.01, TL_A: 0.96, TL_P: 0.95, TL_N: 0.96
Epoch: 6 Loss: 0.38, TL: 1.01, TL_A: 0.96, TL_P: 0.94, TL_N: 0.96
Epoch: 7 Loss: 0.37, TL: 1.02, TL_A: 0.97, TL_P: 0.95, TL_N: 0.96
Epoch: 8 Loss: 0.37, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.97
Epoch: 9 Loss: 0.37, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.97
Epoch: 10 Loss: 0.37, TL: 1.01, TL_A: 0.96, TL_P: 0.95, TL_N: 0.96
Epoch: 11 Loss: 0.37, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.97
Epoch: 12 Loss: 0.37, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.96
Epoch: 13 Loss: 0.37, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.96
Epoch: 14 Loss: 0.37, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.96
Epoch: 15 Loss: 0.37, TL: 1.03, TL_A: 0.95, TL_P: 0.94, TL_N: 0.96
Epoc

Epoch: 124 Loss: 0.22, TL: 1.27, TL_A: 0.96, TL_P: 0.96, TL_N: 0.98
Epoch: 125 Loss: 0.23, TL: 1.35, TL_A: 0.97, TL_P: 0.84, TL_N: 0.99
Epoch: 126 Loss: 0.24, TL: 1.30, TL_A: 0.97, TL_P: 0.84, TL_N: 0.99
Epoch: 127 Loss: 0.23, TL: 1.32, TL_A: 1.00, TL_P: 0.82, TL_N: 0.97
Epoch: 128 Loss: 0.20, TL: 1.28, TL_A: 1.08, TL_P: 0.85, TL_N: 0.88
Epoch: 129 Loss: 0.22, TL: 1.38, TL_A: 1.00, TL_P: 0.80, TL_N: 0.96
Epoch: 130 Loss: 0.21, TL: 1.41, TL_A: 1.00, TL_P: 0.84, TL_N: 0.97
Epoch: 131 Loss: 0.23, TL: 1.30, TL_A: 0.99, TL_P: 0.78, TL_N: 0.95
Epoch: 132 Loss: 0.20, TL: 1.40, TL_A: 1.04, TL_P: 0.84, TL_N: 0.92
Epoch: 133 Loss: 0.27, TL: 1.30, TL_A: 0.84, TL_P: 0.82, TL_N: 1.11
Epoch: 134 Loss: 0.23, TL: 1.31, TL_A: 1.00, TL_P: 0.85, TL_N: 0.98
Epoch: 135 Loss: 0.25, TL: 1.30, TL_A: 0.93, TL_P: 0.83, TL_N: 1.05
Epoch: 136 Loss: 0.24, TL: 1.26, TL_A: 1.01, TL_P: 0.81, TL_N: 0.99
Epoch: 137 Loss: 0.24, TL: 1.32, TL_A: 0.90, TL_P: 0.84, TL_N: 1.04
Epoch: 138 Loss: 0.24, TL: 1.31, TL_A: 0.95, TL_

Epoch: 245 Loss: 0.13, TL: 1.11, TL_A: 0.99, TL_P: 0.98, TL_N: 1.00
Epoch: 246 Loss: 0.13, TL: 1.10, TL_A: 0.99, TL_P: 0.97, TL_N: 0.99
Epoch: 247 Loss: 0.14, TL: 1.01, TL_A: 1.00, TL_P: 0.99, TL_N: 0.99
Epoch: 248 Loss: 0.13, TL: 1.08, TL_A: 0.99, TL_P: 0.98, TL_N: 0.99
Epoch: 249 Loss: 0.13, TL: 1.08, TL_A: 1.02, TL_P: 0.96, TL_N: 0.98
Epoch: 250 Loss: 0.13, TL: 1.08, TL_A: 0.98, TL_P: 1.00, TL_N: 1.00
Epoch: 251 Loss: 0.13, TL: 1.06, TL_A: 0.98, TL_P: 0.97, TL_N: 0.99
Epoch: 252 Loss: 0.13, TL: 1.06, TL_A: 1.00, TL_P: 0.98, TL_N: 0.99
Epoch: 253 Loss: 0.12, TL: 1.09, TL_A: 0.98, TL_P: 0.99, TL_N: 1.00
Epoch: 254 Loss: 0.12, TL: 1.07, TL_A: 0.99, TL_P: 0.99, TL_N: 0.99
Epoch: 255 Loss: 0.12, TL: 1.10, TL_A: 0.99, TL_P: 0.99, TL_N: 0.99
Epoch: 256 Loss: 0.11, TL: 1.08, TL_A: 1.03, TL_P: 0.98, TL_N: 0.97
Epoch: 257 Loss: 0.10, TL: 1.17, TL_A: 1.03, TL_P: 0.93, TL_N: 0.95
Epoch: 258 Loss: 0.11, TL: 1.14, TL_A: 1.00, TL_P: 0.98, TL_N: 0.99
Epoch: 259 Loss: 0.12, TL: 1.08, TL_A: 1.00, TL_

Epoch: 366 Loss: -0.02, TL: 1.24, TL_A: 0.95, TL_P: 0.97, TL_N: 0.98
Epoch: 367 Loss: -0.03, TL: 1.31, TL_A: 1.04, TL_P: 0.88, TL_N: 0.93
Epoch: 368 Loss: -0.01, TL: 1.31, TL_A: 1.04, TL_P: 0.78, TL_N: 0.94
Epoch: 369 Loss: 0.02, TL: 1.25, TL_A: 0.93, TL_P: 0.88, TL_N: 1.07
Epoch: 370 Loss: -0.01, TL: 1.30, TL_A: 1.07, TL_P: 0.80, TL_N: 0.93
Epoch: 371 Loss: -0.03, TL: 1.34, TL_A: 0.96, TL_P: 0.80, TL_N: 0.93
Epoch: 372 Loss: -0.05, TL: 1.43, TL_A: 1.08, TL_P: 0.76, TL_N: 0.86
Epoch: 373 Loss: 0.01, TL: 1.29, TL_A: 0.93, TL_P: 0.77, TL_N: 1.02
Epoch: 374 Loss: 0.01, TL: 1.32, TL_A: 1.00, TL_P: 0.76, TL_N: 1.00
Epoch: 375 Loss: -0.06, TL: 1.38, TL_A: 1.11, TL_P: 0.78, TL_N: 0.83
Epoch: 376 Loss: -0.03, TL: 1.35, TL_A: 1.10, TL_P: 0.70, TL_N: 0.88
Epoch: 377 Loss: -0.03, TL: 1.28, TL_A: 0.97, TL_P: 0.97, TL_N: 0.99
Epoch: 378 Loss: -0.00, TL: 1.22, TL_A: 1.03, TL_P: 0.77, TL_N: 0.93
Epoch: 379 Loss: -0.00, TL: 1.25, TL_A: 0.90, TL_P: 0.91, TL_N: 1.06
Epoch: 380 Loss: -0.03, TL: 1.23, TL_

Epoch: 485 Loss: -0.13, TL: 1.21, TL_A: 1.00, TL_P: 0.89, TL_N: 0.95
Epoch: 486 Loss: -0.11, TL: 1.38, TL_A: 0.94, TL_P: 0.80, TL_N: 1.05
Epoch: 487 Loss: -0.15, TL: 1.38, TL_A: 1.05, TL_P: 0.86, TL_N: 0.94
Epoch: 488 Loss: -0.12, TL: 1.27, TL_A: 1.03, TL_P: 0.77, TL_N: 0.96
Epoch: 489 Loss: -0.13, TL: 1.36, TL_A: 0.98, TL_P: 0.80, TL_N: 0.96
Epoch: 490 Loss: -0.12, TL: 1.44, TL_A: 0.91, TL_P: 0.73, TL_N: 1.03
Epoch: 491 Loss: -0.12, TL: 1.30, TL_A: 0.95, TL_P: 0.89, TL_N: 1.03
Epoch: 492 Loss: -0.06, TL: 1.31, TL_A: 0.83, TL_P: 0.72, TL_N: 1.16
Epoch: 493 Loss: -0.12, TL: 1.36, TL_A: 0.97, TL_P: 0.78, TL_N: 1.01
Epoch: 494 Loss: -0.14, TL: 1.37, TL_A: 0.96, TL_P: 0.90, TL_N: 1.03
Epoch: 495 Loss: -0.16, TL: 1.42, TL_A: 1.03, TL_P: 0.85, TL_N: 0.94
Epoch: 496 Loss: -0.10, TL: 1.24, TL_A: 0.94, TL_P: 0.74, TL_N: 1.00
Epoch: 497 Loss: -0.17, TL: 1.35, TL_A: 1.05, TL_P: 0.87, TL_N: 0.91
Epoch: 498 Loss: -0.13, TL: 1.40, TL_A: 0.97, TL_P: 0.75, TL_N: 1.00
Epoch: 499 Loss: -0.12, TL: 1.25, 

Epoch: 604 Loss: -0.21, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 605 Loss: -0.21, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 606 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 607 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 608 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 609 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 610 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 611 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 0.99, TL_N: 1.00
Epoch: 612 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 613 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 614 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 615 Loss: -0.23, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 616 Loss: -0.23, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 617 Loss: -0.23, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 618 Loss: -0.23, TL: 1.00, 

Epoch: 723 Loss: -0.36, TL: 1.19, TL_A: 0.99, TL_P: 0.97, TL_N: 1.00
Epoch: 724 Loss: -0.36, TL: 1.23, TL_A: 1.03, TL_P: 0.91, TL_N: 0.97
Epoch: 725 Loss: -0.37, TL: 1.27, TL_A: 1.00, TL_P: 0.92, TL_N: 1.00
Epoch: 726 Loss: -0.36, TL: 1.26, TL_A: 0.97, TL_P: 0.95, TL_N: 1.04
Epoch: 727 Loss: -0.35, TL: 1.14, TL_A: 1.00, TL_P: 0.89, TL_N: 0.98
Epoch: 728 Loss: -0.36, TL: 1.21, TL_A: 1.02, TL_P: 0.89, TL_N: 0.97
Epoch: 729 Loss: -0.36, TL: 1.26, TL_A: 0.96, TL_P: 0.93, TL_N: 1.04
Epoch: 730 Loss: -0.37, TL: 1.22, TL_A: 1.02, TL_P: 0.94, TL_N: 1.00
Epoch: 731 Loss: -0.36, TL: 1.17, TL_A: 1.00, TL_P: 0.93, TL_N: 0.99
Epoch: 732 Loss: -0.35, TL: 1.09, TL_A: 0.99, TL_P: 0.93, TL_N: 0.98
Epoch: 733 Loss: -0.37, TL: 1.20, TL_A: 1.00, TL_P: 0.95, TL_N: 1.01
Epoch: 734 Loss: -0.38, TL: 1.27, TL_A: 1.01, TL_P: 0.93, TL_N: 1.00
Epoch: 735 Loss: -0.36, TL: 1.25, TL_A: 0.99, TL_P: 0.89, TL_N: 1.00
Epoch: 736 Loss: -0.34, TL: 1.14, TL_A: 0.95, TL_P: 0.88, TL_N: 1.03
Epoch: 737 Loss: -0.39, TL: 1.31, 

Epoch: 842 Loss: -0.45, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 843 Loss: -0.45, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 844 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 845 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 846 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 847 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 848 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 849 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 850 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 851 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 852 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 853 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 854 Loss: -0.47, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 855 Loss: -0.47, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 856 Loss: -0.47, TL: 1.00, 

Epoch: 961 Loss: -0.57, TL: 1.01, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 962 Loss: -0.57, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 963 Loss: -0.57, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 964 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 965 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 966 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 967 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 968 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 969 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 970 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 971 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 972 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 973 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 974 Loss: -0.59, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 975 Loss: -0.59, TL: 1.00, 

In [48]:
recall

0.21

In [49]:
_[:20]

['327681:324658|327985:0.9428757652640343,324056:0.9425551109015942,343513:0.9424966424703598,318846:0.9424221701920033,352297:0.9420154206454754,318845:0.9420088715851307,325924:0.9417591020464897,400354:0.9417415969073772,394703:0.9416719935834408,343328:0.9415632486343384,320640:0.9414856433868408,280457:0.9414570070803165,394104:0.9410785920917988,342369:0.9410055316984653,336435:0.9410013109445572,303904:0.9409882128238678,330650:0.9409813396632671,315120:0.9408961646258831,233093:0.9408209845423698,345982:0.9406372345983982,311065:0.9406181387603283,350844:0.9406060986220837,357210:0.9405073411762714,338351:0.9405035898089409,396638:0.9402814470231533,356356:0.940276350826025,344444:0.9402624070644379,354235:0.9402533769607544,361530:0.9402143992483616',
 '360457:362252|364552:0.9639727622270584,354118:0.9556278437376022,353222:0.9505909606814384,367431:0.9504093267023563,338252:0.9497708156704903,343568:0.949703074991703,396905:0.9494970850646496,377371:0.9488717056810856,353496

In [50]:
# recall, exported_rank, debug = experiment.evaluate_validation_test(experiment, retrieval, verbose, 
#                                                         encoded_anchor, issues_by_buckets, evaluate_validation_test)
# test_vectorized, queries_test_vectorized, annoy, X_test, distance_test, indices_test = debug
# "recall@25 last epoch:", recall

### Retrieval evaluation

In [51]:
print("Total of queries:", len(retrieval.test))

Total of queries: 4641


#### Getting the model trained

In [52]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

'deepQL_weights_1000_feature_1000epochs_64batch(eclipse)'

In [53]:
model = encoded_anchor
# model = experiment.get_model_vectorizer(path=SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

In [54]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 1682)         0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 20)           0                                            
__________________________________________________________________________________________________
desc_segme

In [55]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 0, model, issues_by_buckets, 
                                                                   bug_train_ids, method='bert')

In [56]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/eclipse/exported_rank_deepQL_weights_1000.txt'

In [57]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [58]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.15,
 '2 - recall_at_10': 0.18,
 '3 - recall_at_15': 0.19,
 '4 - recall_at_20': 0.21,
 '5 - recall_at_25': 0.21}